### Data munging
Creates a csv file from image info.

In [22]:
# general imports
import os, glob, fnmatch
import pandas as pd
import numpy as np

# image processing imports
import cv2

---
## Create dataframe
Creates a dataframe from RGB, HSV, image height & width image data using OpenCV methods and `.shape`.

In [6]:
# create list of all the image filepaths
images = []
for root, dirnames, filenames in os.walk('/Users/VanessaG/Desktop/pizza_class_data/'):
    for filename in fnmatch.filter(filenames, '*.jpg'):
        images.append(os.path.join(root, filename))

In [7]:
# create lists for dataframe of image info - rgb, hsv, image height & width
data = []
for img in images:
    image = cv2.imread(img)
    data.append(cv2.normalize(image, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F))
    
rgb_means = []
hsv_means = []
img_height = []
img_width = [] 

for i in data:
    means = cv2.mean(cv2.cvtColor(i, cv2.COLOR_BGR2RGB))[:3]
    rgb_means.append(means)
    means2 = cv2.mean(cv2.cvtColor(i, cv2.COLOR_BGR2HSV))[:3]
    hsv_means.append(means2)
    img_height.append(i.shape[0])
    img_width.append(i.shape[1])

In [8]:
# create dataframes from the lists above
df1 = pd.DataFrame(rgb_means, columns=['red', 'green', 'blue'])
df2 = pd.DataFrame(hsv_means, columns=['hue', 'sat', 'val'])
df = pd.concat([df1, df2], axis=1)

In [9]:
# just nice to have - used to display images in EDA
df['full_path'] = images

In [10]:
# 0 is not pizza, 1 is pizza
df['label'] = df.full_path.map(lambda x: 0 if '/not_pizza/' in x else 1)

In [11]:
# basically confirming labels are correct
df['short_path'] = df['full_path'].replace({'/Users/VanessaG/Desktop/pizza_class_data/': ''}, regex=True)

In [12]:
# add in width, height, total pixels and re-order columns
df['img_height'] = img_height
df['img_width'] = img_width
df['total_px'] = df.img_height * df.img_width
df = df[['label', 'red', 'green', 'blue','hue', 'sat', 'val', 'img_height', 'img_width', 'total_px', 'short_path', 'full_path']]

In [13]:
# save csv
df.to_csv('image_info.csv')